### 3.1 Predict Messages

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)

# 가상의 대화 설정
messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Korean."
    ),
    AIMessage(content="안녕! 나는 챗봇이야."),
    HumanMessage(content="What is the distance between Korea and Japan? Also, what is your name?"),
]

chat.predict_messages(messages)

AIMessage(content='한국과 일본 사이의 거리는 약 800km입니다. 제 이름은 지리 전문가 챗봇이에요.')

### 3.2 Prompt Templates

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}?",
)

prompt = template.format(country_a="Korea", country_b="Vietnam")

chat.predict(prompt)

'The distance between Korea and Vietnam is approximately 2,500 kilometers (1,550 miles) when measured in a straight line.'

In [3]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "안녕! 나는 {name}이야."),
        ("human", "What is the distance between {country_a} and {country_b}? also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language="Korean",
    name="chatbot",
    country_a="Korea",
    country_b="France",
)

chat.predict_messages(prompt)

AIMessage(content='한국과 프랑스 사이의 거리는 대략 9000km입니다. 제 이름은 AI 어시스턴트입니다. 어떻게 도와드릴까요?')

### 3.3 OutputParser and LCEL(LangChain Expression Language)

In [4]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    # must define parse function
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse('hi,how,are,you')

['hi', 'how', 'are', 'you']

In [5]:
# without LCEL

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
        ("human", "{question}."),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [6]:
# with LCEL
# make a chain of chat model, OutputParser and template.

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
        ("human", "{question}."),
    ]
)

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle']

### 3.4 Chaining Chains

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# streaming: LLM의 응답이 생성되는걸 볼 수 있게 해줌
# StreamingStdOutCallbackHandler: console에서 응답의 진행을 볼 수 있음
chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [8]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

In [9]:
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon garam masala
- 1 teaspoon ground coriander
- 1 teaspoon chili powder (adjust to taste)
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, and chili powder. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best r

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare the tofu or paneer as a substitute for the chicken in this recipe:\n\nTofu:\n1. Use firm or extra firm tofu for this recipe. Press the tofu to remove excess water by wrapping it in a clean kitchen towel and placing a heavy object on top for about 15-20 minutes.\n2. Cut the tofu into bite-sized cubes and proceed with marinating it in the yogurt and spice mixture as you would with the chicken. Make sure to coat the tofu pieces well and refrigerate for at least 1 hour.\n3. Instead of baking, you can pan-fry the marinated tofu in a bit of oil until it is golden brown and slightly crispy on the outside.\n\nPaneer:\n1. If you choose to use paneer, you can skip the marinating step as paneer does not need to be marinated like tofu. Simply cut the paneer into bite-sized cubes.\n2. You can lightly pan-fry the panee

### 4.1 FewShotPromptTemplate

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

caht = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# AI에게 줄 답변 예제들
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [11]:
# Before I give examples to Language Model(LM)
chat.predict("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of around 67 million people. The official language is French, and the currency is the Euro.

France has a diverse landscape, including mountains, forests, beaches, and vineyards. It is known for its wine production, particularly in regions such as Bordeaux, Burgundy, and Champagne.

France has a long history of art and literature, with famous artists such as Claude Monet, Vincent van Gogh, and Henri Matisse, and writers such as Victor Hugo, Albert Camus, and Marcel Proust.

France is a founding member of the United Nations, NATO, and the European Union. It is also a permanent member of the United Nations Security Council. The country has 

'France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of around 67 million people. The official language is French, and the currency is the Euro.\n\nFrance has a diverse landscape, including mountains, forests, beaches, and vineyards. It is known for its wine production, particularly in regions such as Bordeaux, Burgundy, and Champagne.\n\nFrance has a long history of art and literature, with famous artists such as Claude Monet, Vincent van Gogh, and Henri Matisse, and writers such as Victor Hugo, Albert Camus, and Marcel Proust.\n\nFrance is a founding member of the United Nations, NATO, and the European Union. It is also a permanent member of the United Nations Security Council. The cou

In [12]:
# {}안의 변수 이름은 예제와 같게 설정
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you konw about {country}?", # 앞의 예제를 보고나서 AI가 답변해야 할 질문
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
        Here is what I know:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

### 4.2 FewShotChatMessagePromptTemplate